In [1]:
# -----------------------------------------------
# EXO 6 – Interactive Visual Analytics with Folium
# -----------------------------------------------
!pip install pandas folium

import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MarkerCluster
from math import sin, cos, sqrt, atan2, radians

# Charger le dataset (dataset_part_2.csv depuis Exo 3)
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv")

# Ajouter une colonne pour la couleur du marker selon le succès
df['marker_color'] = df['Class'].apply(lambda x: 'green' if x==1 else 'red')

# Coordonnées approximatives du NASA Johnson Space Center
nasa_coordinate = [29.5597, -95.0831]

# Créer la carte centrée sur NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# MarkerCluster pour grouper les markers
marker_cluster = MarkerCluster()

# Ajouter un cercle et un marker pour NASA JSC
circle = folium.Circle(
    location=nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    location=nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>NASA JSC</b></div>'
    )
)

site_map.add_child(circle)
site_map.add_child(marker)

# Ajouter un marker pour chaque site de lancement
launch_sites = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates()

for index, site in launch_sites.iterrows():
    coord = [site['Latitude'], site['Longitude']]
    folium.Circle(
        location=coord,
        radius=1000,
        color='#0000FF',
        fill=True
    ).add_child(folium.Popup(site['LaunchSite'])).add_to(site_map)
    
    folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#0000FF;"><b>{site["LaunchSite"]}</b></div>'
        )
    ).add_to(site_map)

# Ajouter un marker pour chaque lancement avec succès/échec
for index, record in df.iterrows():
    folium.Marker(
        location=[record['Latitude'], record['Longitude']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Booster: {record['BoosterVersion']}\nOutcome: {record['Outcome']}"
    ).add_to(marker_cluster)

site_map.add_child(marker_cluster)

# Fonction pour calculer la distance entre deux points
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # km
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

# Exemple : Marquer la côte la plus proche du site de lancement CCAFS SLC 40
launch_site_coord = [28.561857, -80.577366]
coastline_coord = [28.56367, -80.57163]  # Exemple manuel
distance = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                              coastline_coord[0], coastline_coord[1])

# Ajouter marker pour la côte
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>'
    )
)
distance_marker.add_to(site_map)

# Tracer une ligne entre le site de lancement et la côte
lines = folium.PolyLine(locations=[launch_site_coord, coastline_coord], weight=2, color='blue')
site_map.add_child(lines)

# Afficher la carte interactive
site_map


In [4]:
# -----------------------------------------------
# SpaceX Launch Sites Global Map for Diapo
# -----------------------------------------------

# Installer les packages dans le kernel actif
import sys
!{sys.executable} -m pip install --quiet pandas folium

# Importer les bibliothèques
import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MarkerCluster

# Charger le dataset
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
df = pd.read_csv(url)

# Ajouter une colonne pour la couleur du marker selon succès
df['marker_color'] = df['Class'].apply(lambda x: 'green' if x == 1 else 'red')

# Créer la carte centrée globalement
site_map = folium.Map(location=[20, 0], zoom_start=2, tiles='cartodbpositron')

# Ajouter un MarkerCluster pour grouper les markers
marker_cluster = MarkerCluster()

# Ajouter un marker pour chaque lancement
for _, record in df.iterrows():
    folium.Marker(
        location=[record['Latitude'], record['Longitude']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Site: {record['LaunchSite']}\nBooster: {record['BoosterVersion']}\nOutcome: {record['Outcome']}"
    ).add_to(marker_cluster)

# Ajouter le MarkerCluster à la carte
site_map.add_child(marker_cluster)

# Ajouter des markers pour les sites de lancement uniques
launch_sites = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates()
for _, site in launch_sites.iterrows():
    folium.Marker(
        location=[site['Latitude'], site['Longitude']],
        icon=DivIcon(
            icon_size=(150,36),
            html=f'<div style="font-size:12px; color:blue;"><b>{site["LaunchSite"]}</b></div>'
        )
    ).add_to(site_map)

# Sauvegarder la carte en HTML pour capture
site_map.save("launch_sites_map.html")

# Afficher la carte interactive directement dans le notebook
site_map


In [5]:
# -----------------------------------------------
# SpaceX Launch Outcomes Map
# -----------------------------------------------

import sys
!{sys.executable} -m pip install --quiet pandas folium

import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Charger le dataset
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
df = pd.read_csv(url)

# Ajouter une colonne pour la couleur du marker selon succès
df['marker_color'] = df['Class'].apply(lambda x: 'green' if x == 1 else 'red')

# Carte centrée globalement
outcome_map = folium.Map(location=[20,0], zoom_start=2, tiles='cartodbpositron')

# MarkerCluster pour grouper les markers
marker_cluster = MarkerCluster()

# Ajouter markers pour chaque lancement
for _, record in df.iterrows():
    folium.Marker(
        location=[record['Latitude'], record['Longitude']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Site: {record['LaunchSite']}\nOutcome: {record['Outcome']}"
    ).add_to(marker_cluster)

outcome_map.add_child(marker_cluster)

# Sauvegarder en HTML pour screenshot
outcome_map.save("launch_outcomes_map.html")

# Afficher la carte dans le notebook
outcome_map


In [6]:
# -----------------------------------------------
# SpaceX Launch Site Proximity Map
# -----------------------------------------------

import sys
!{sys.executable} -m pip install --quiet pandas folium geopy

import pandas as pd
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Charger le dataset
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
df = pd.read_csv(url)

# Fonction pour calculer la distance entre deux points (en km)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Choisir un site de lancement (exemple: CCAFS SLC 40)
launch_site_coord = [28.561857, -80.577366]  # Kennedy Space Center

# Définir des proximités (exemple: coastline, highway, railway)
proximities = {
    "Coastline": [28.56367, -80.57163],
    "Highway": [28.5625, -80.5850],
    "Railway": [28.5590, -80.5800]
}

# Créer la carte centrée sur le site de lancement
proximity_map = folium.Map(location=launch_site_coord, zoom_start=14)

# Ajouter marker pour le site de lancement
folium.Marker(
    location=launch_site_coord,
    icon=DivIcon(
        icon_size=(150,36),
        html='<div style="font-size:12px; color:red;"><b>Launch Site</b></div>'
    )
).add_to(proximity_map)

# Ajouter markers pour les proximités et afficher distance
for name, coord in proximities.items():
    distance = calculate_distance(launch_site_coord[0], launch_site_coord[1], coord[0], coord[1])
    folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,36),
            html=f'<div style="font-size:12px; color:blue;"><b>{name}: {distance:.2f} km</b></div>'
        )
    ).add_to(proximity_map)
    # Tracer une ligne entre le site et la proximité
    folium.PolyLine(locations=[launch_site_coord, coord], color='green', weight=2).add_to(proximity_map)

# Sauvegarder la carte en HTML
proximity_map.save("launch_site_proximities_map.html")

# Afficher la carte interactive
proximity_map
